In [1]:
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import sklearn.preprocessing
import sklearn.metrics
from data_split_tune_utils import X_y_site_split
from CNN_utils import split_sizes_site, split_data, pad_stack_splits, get_monitorData_indices, r2, get_nonConst_vars, train_CNN
from CNN_architecture import CNN1, CNN2

np.random.seed(1)
torch.manual_seed(1)

### read in train, val, and test
train = pd.read_csv('../data/trainV_ridgeImp.csv')
val = pd.read_csv('../data/valV_ridgeImp.csv')
test = pd.read_csv('../data/testV_ridgeImp.csv')

### delete sites from datasets where all monitor outputs are NaN
train_sites_all_nan_df = pd.DataFrame(np.isnan(train.groupby('site').sum()['MonitorData']))
train_sites_to_delete = list(train_sites_all_nan_df[train_sites_all_nan_df['MonitorData'] == True].index)
train = train[~train['site'].isin(train_sites_to_delete)]

val_sites_all_nan_df = pd.DataFrame(np.isnan(val.groupby('site').sum()['MonitorData']))
val_sites_to_delete = list(val_sites_all_nan_df[val_sites_all_nan_df['MonitorData'] == True].index)
val = val[~val['site'].isin(val_sites_to_delete)]

test_sites_all_nan_df = pd.DataFrame(np.isnan(test.groupby('site').sum()['MonitorData']))
test_sites_to_delete = list(test_sites_all_nan_df[test_sites_all_nan_df['MonitorData'] == True].index)
test = test[~test['site'].isin(test_sites_to_delete)]

### split train, val, and test into x, y, and sites
train_x, train_y, train_sites = X_y_site_split(train, y_var_name='MonitorData', site_var_name='site')
val_x, val_y, val_sites = X_y_site_split(val, y_var_name='MonitorData', site_var_name='site')
test_x, test_y, test_sites = X_y_site_split(test, y_var_name='MonitorData', site_var_name='site')

### get dataframes with non-constant features only
nonConst_vars = get_nonConst_vars(train, site_var_name='site', y_var_name='MonitorData', cutoff=1000)
train_x_nonConst = train_x.loc[:, nonConst_vars]
val_x_nonConst = val_x.loc[:, nonConst_vars]
test_x_nonConst = test_x.loc[:, nonConst_vars]

### standardize all features
standardizer_all = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
train_x_std_all = standardizer_all.fit_transform(train_x)
val_x_std_all = standardizer_all.transform(val_x)
test_x_std_all = standardizer_all.transform(test_x)

### standardize non-constant features
standardizer_nonConst = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
train_x_std_nonConst = standardizer_nonConst.fit_transform(train_x_nonConst)
val_x_std_nonConst = standardizer_nonConst.transform(val_x_nonConst)
test_x_std_nonConst = standardizer_nonConst.transform(test_x_nonConst)




### get split sizes for TRAIN data (splitting by site)
train_split_sizes = split_sizes_site(train_sites.values)

### get tuples by site
train_x_std_tuple_nonConst = split_data(torch.from_numpy(train_x_std_nonConst).float(), train_split_sizes, dim = 0)
train_x_std_tuple = split_data(torch.from_numpy(train_x_std_all).float(), train_split_sizes, dim = 0)
train_y_tuple = split_data(torch.from_numpy(train_y.values), train_split_sizes, dim = 0)

### get site sequences stacked into matrix to go through CNN
train_x_std_stack_nonConst = pad_stack_splits(train_x_std_tuple_nonConst, np.array(train_split_sizes), 'x')
train_x_std_stack_nonConst = Variable(torch.transpose(train_x_std_stack_nonConst, 1, 2))


### get split sizes for VALIDATION data (splitting by site)
val_split_sizes = split_sizes_site(val_sites.values)

### get tuples by site
val_x_std_tuple_nonConst = split_data(torch.from_numpy(val_x_std_nonConst).float(), val_split_sizes, dim = 0)
val_x_std_tuple = split_data(torch.from_numpy(val_x_std_all).float(), val_split_sizes, dim = 0)
val_y_tuple = split_data(torch.from_numpy(val_y.values), val_split_sizes, dim = 0)

### get site sequences stacked into matrix to go through CNN
val_x_std_stack_nonConst = pad_stack_splits(val_x_std_tuple_nonConst, np.array(val_split_sizes), 'x')
val_x_std_stack_nonConst = Variable(torch.transpose(val_x_std_stack_nonConst, 1, 2))


### get split sizes for TEST data (splitting by site)
test_split_sizes = split_sizes_site(test_sites.values)

### get tuples by site
test_x_std_tuple_nonConst = split_data(torch.from_numpy(test_x_std_nonConst).float(), test_split_sizes, dim = 0)
test_x_std_tuple = split_data(torch.from_numpy(test_x_std_all).float(), test_split_sizes, dim = 0)
test_y_tuple = split_data(torch.from_numpy(test_y.values), test_split_sizes, dim = 0)

### get site sequences stacked into matrix to go through CNN
test_x_std_stack_nonConst = pad_stack_splits(test_x_std_tuple_nonConst, np.array(test_split_sizes), 'x')
test_x_std_stack_nonConst = Variable(torch.transpose(test_x_std_stack_nonConst, 1, 2))

In [4]:
num_epochs = 5
batch_size = 32
input_size_conv = train_x_std_nonConst.shape[1]
input_size_full = train_x_std_all.shape[1]

# CNN and optimizer hyper-parameters to test
hidden_size_conv_list = [25, 50, 75]
kernel_size_list = [3, 5]
padding_list = [1, 2]
hidden_size_full_list = [50, 100, 150]
dropout_full_list = [0.1, 0.3, 0.5]
hidden_size_combo_list = [50, 100, 150]
dropout_combo_list = [0.1, 0.3, 0.5]
lr_list = [0.1, 0.01]
weight_decay_list = [0.001, 0.0001, 0.00001]

# Loss function
mse_loss = torch.nn.MSELoss(size_average=True)

In [ ]:
best_val_r2 = -np.inf
for hidden_size_conv in hidden_size_conv_list:
    for kernel_size, padding in zip(kernel_size_list, padding_list):
        for hidden_size_full in hidden_size_full_list:
            for dropout_full in dropout_full_list:
                for hidden_size_combo in hidden_size_combo_list:
                    for dropout_combo in dropout_combo_list:
                        for lr in lr_list:
                            for weight_decay in weight_decay_list:
                                
                                # instantiate CNN
                                cnn = CNN1(input_size_conv, hidden_size_conv, kernel_size, padding, input_size_full, hidden_size_full, 
                                          dropout_full, hidden_size_combo, dropout_combo)
                                
                                # instantiate optimizer
                                optimizer = torch.optim.Adam(cnn.parameters(), lr=lr, weight_decay=weight_decay)
                                
                                print('Hidden size conv: ' + str(hidden_size_conv))
                                print('Kernel size: ' + str(kernel_size))
                                print('Hidden size full: ' + str(hidden_size_full))
                                print('Dropout full: ' + str(dropout_full))
                                print('Hidden size combo: ' + str(hidden_size_combo))
                                print('Dropout combo: ' + str(dropout_combo))
                                print('Learning rate: ' + str(lr))
                                print('Weight decay: ' + str(weight_decay))

                                train_CNN(train_x_std_stack_nonConst, train_x_std_tuple, train_y_tuple, cnn, optimizer, mse_loss, num_epochs, batch_size)
                                
                                val_r2 = r2(cnn, batch_size, val_x_std_stack_nonConst, val_x_std_tuple, val_y_tuple)
                                print('Validation R^2: ' + str(val_r2))
                                print()
                                print()
                                x
                                if val_r2 > best_val_r2:
                                    best_val_r2 = val_r2
                                    best_hidden_size_conv = hidden_size_conv
                                    best_kernel_size = kernel_size
                                    best_hidden_size_full = hidden_size_full
                                    best_dropout_full = dropout_full
                                    best_hidden_size_combo = hidden_size_combo
                                    best_dropout_combo = dropout_combo
                                    best_lr = lr
                                    best_weight_decay = weight_decay
                                    
print('Best validation R^2: ' + str(best_val_r2))
print('Best hidden size conv: ' + str(best_hidden_size_conv))
print('Best kernel size: ' + str(best_kernel_size))
print('Best hidden size full: ' + str(best_hidden_size_full))
print('Best dropout full: ' + str(best_dropout_full))
print('Best hidden size combo: ' + str(best_hidden_size_combo))
print('Best dropout combo: ' + str(best_dropout_combo))
print('Best learning rate: ' + str(best_lr))
print('Best weight decay: ' + str(best_weight_decay))                   

Hidden size conv: 25
Kernel size: 3
Hidden size full: 50
Dropout full: 0.1
Hidden size combo: 50
Dropout combo: 0.1
Learning rate: 0.1
Weight decay: 0.001


In [35]:
r2(cnn, batch_size, test_x_std_stack_nonConst, test_x_std_tuple, test_y_tuple)

torch.Size([32, 22, 6210])
32
here r2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
torch.Size([13, 22, 6210])
torch.Size([12, 22, 6210])
12


0.058542578575068727

In [12]:
num_epochs = 21
batch_size = 64
input_size_conv = train_x_std_nonConst.shape[1]
input_size_full = train_x_std_all.shape[1]

# CNN and optimizer hyper-parameters to test
hidden_size_conv_list = [25, 50, 75]
kernel_size_list = [3, 5]
padding_list = [1, 2]
hidden_size_full_list = [50, 100, 150]
dropout_full_list = [0.1, 0.3, 0.5]
hidden_size2_full_list = [50, 100, 150]
dropout2_full_list = [0.1, 0.3, 0.5]
lr_list = [0.1, 0.01]
weight_decay_list = [0.001, 0.0001, 0.00001]

# Loss function
mse_loss = torch.nn.MSELoss(size_average=True)

In [13]:
best_val_r2 = -np.inf
for hidden_size_conv in hidden_size_conv_list:
    for kernel_size, padding in zip(kernel_size_list, padding_list):
        for hidden_size_full in hidden_size_full_list:
            for dropout_full in dropout_full_list:
                for hidden_size2_full in hidden_size2_full_list:
                    for dropout2_full in dropout2_full_list:
                        for lr in lr_list:
                            for weight_decay in weight_decay_list:
                                
                                # instantiate CNN
                                cnn = CNN2(input_size_conv, hidden_size_conv, kernel_size, padding, input_size_full, hidden_size_full, 
                                          dropout_full, hidden_size2_full, dropout2_full)
                                
                                # instantiate optimizer
                                optimizer = torch.optim.Adam(cnn.parameters(), lr=lr, weight_decay=weight_decay)
                                
                                print('Hidden size conv: ' + str(hidden_size_conv))
                                print('Kernel size: ' + str(kernel_size))
                                print('Hidden size full: ' + str(hidden_size_full))
                                print('Dropout full: ' + str(dropout_full))
                                print('Hidden size 2 full: ' + str(hidden_size2_full))
                                print('Dropout 2 full: ' + str(dropout2_full))
                                print('Learning rate: ' + str(lr))
                                print('Weight decay: ' + str(weight_decay))

                                train_CNN(train_x_std_stack_nonConst, train_x_std_tuple, train_y_tuple, cnn, optimizer, mse_loss, num_epochs, batch_size)
                                
                                val_r2 = r2(cnn, batch_size, val_x_std_stack_nonConst, val_x_std_tuple, val_y_tuple)
                                print('Validation R^2: ' + str(val_r2))
                                print()
                                print()
                                
                                if val_r2 > best_val_r2:
                                    best_val_r2 = val_r2
                                    best_hidden_size_conv = hidden_size_conv
                                    best_kernel_size = kernel_size
                                    best_hidden_size_full = hidden_size_full
                                    best_dropout_full = dropout_full
                                    best_hidden_size2_full = hidden_size2_full
                                    best_dropout2_full = dropout2_full
                                    best_lr = lr
                                    best_weight_decay = weight_decay
                                    
print('Best validation R^2: ' + str(best_val_r2))
print('Best hidden size conv: ' + str(best_hidden_size_conv))
print('Best kernel size: ' + str(best_kernel_size))
print('Best hidden size full: ' + str(best_hidden_size_full))
print('Best dropout full: ' + str(best_dropout_full))
print('Best hidden size 2 full: ' + str(best_hidden_size2_full))
print('Best dropout 2 full: ' + str(best_dropout2_full))
print('Best learning rate: ' + str(best_lr))
print('Best weight decay: ' + str(best_weight_decay))                    

Hidden size conv: 25
Kernel size: 3
Hidden size full: 50
Dropout full: 0.1
Hidden size 2 full: 50
Dropout 2 full: 0.1
Learning rate: 0.1
Weight decay: 0.001
0
torch.Size([25, 6210])
torch.Size([900])

   12
   18
   30
   36
   42
   48
   54
   60
   66
   80
   84
   90
   96
  102
  108
  114
  120
  132
  138
  150
  162
  183
  186
  198
  204
  210
  216
  222
  228
  234
  240
  246
  252
  258
  264
  270
  277
  282
  300
  315
  319
  330
  342
  348
  360
  366
  372
  378
  384
  390
  396
  402
  408
  414
  420
  426
  438
  441
  444
  450
  456
  462
  466
  468
  480
  486
  492
  498
  504
  510
  516
  522
  528
  534
  540
  552
  558
  564
  570
  576
  582
  588
  594
  600
  607
  630
  636
  642
  648
  672
  703
  708
  714
  726
  732
  742
  745
  756
  762
  768
  774
  780
  786
  792
  798
  804
  810
  822
  828
  834
  840
  846
  852
  858
  864
  870
  876
  882
  888
  895
  906
  913
  942
  948
  954
  960
  966
  973
  978
  984
  990
  996
 1002
 


 5569
 5570
 5571
 5572
 5573
 5574
 5575
 5576
 5577
 5578
 5579
 5580
 5581
 5582
 5583
 5584
 5585
 5586
 5587
 5588
 5589
 5590
 5591
 5592
 5593
 5594
 5595
 5596
 5597
 5598
 5599
 5600
 5601
 5602
 5603
 5604
 5605
 5606
 5607
 5608
 5609
 5610
 5611
 5612
 5613
 5614
 5615
 5616
 5617
 5618
 5619
 5620
 5621
 5622
 5623
 5624
 5625
 5626
 5627
 5628
 5629
 5630
 5631
 5632
 5633
 5634
 5635
 5636
 5637
 5638
 5639
 5640
 5641
 5642
 5643
 5644
 5645
 5646
 5647
 5648
 5649
 5650
 5651
 5652
 5653
 5654
 5655
 5656
 5657
 5658
 5659
 5660
 5661
 5662
 5663
 5664
 5665
 5666
 5667
 5668
 5669
 5670
 5671
 5672
 5673
 5674
 5675
 5676
 5677
 5678
 5679
 5680
 5681
 5682
 5683
 5684
 5685
 5686
 5687
 5688
 5689
 5690
 5691
 5692
 5693
 5694
 5695
 5696
 5697
 5698
 5699
 5700
 5701
 5702
 5703
 5704
 5705
 5706
 5707
 5708
 5709
 5710
 5711
 5712
 5713
 5714
 5715
 5716
 5717
 5718
 5719
 5720
 5721
 5722
 5723
 5724
 5725
 5726
 5727
 5728
 5729
 5730
 5731
 5732
 5733
 5734
 57

NameError: name 'x' is not defined